In [1]:
import pandas as pd

import pandas as pd
from src.data.ingestion import load_tickets

In [2]:
from src.data.graph_rag_ingest import build_support_graph
from src.data.graph_rag_query import top_solutions_for_issue,graph_rag_candidates

In [3]:
json_data = load_tickets("../data/raw/support_tickets.json")
Data_tickets = json_data[0]
Data_tickets[:1]

[Ticket(ticket_id='TK-2024-000001', created_at=datetime.datetime(2023, 11, 2, 12, 30, 10, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2023, 11, 2, 15, 30, 46, tzinfo=datetime.timezone.utc), customer_id='CUST-02387', customer_tier='starter', organization_id='ORG-234', product='CloudBackup Enterprise', product_version='4.5.10', product_module='encryption_layer', category='Feature Request', subcategory='Documentation', priority='critical', severity='P2', channel='portal', subject='Request: Add bulk operation support to CloudBackup Enterprise', description='We would like to request a feature for CloudBackup Enterprise that allows bulk operations. Currently, we have to process items one by one, which is time-consuming. Having bulk support would greatly improve our workflow efficiency.', error_logs='', stack_trace='', customer_sentiment='frustrated', previous_tickets=9, resolution='Issue resolved by updating configuration settings. Changed timeout values from 30s to 120s in c

In [4]:
support_graph = build_support_graph(Data_tickets[:])

print("nodes:", support_graph.number_of_nodes())
print("edges:", support_graph.number_of_edges())

print(top_solutions_for_issue(support_graph, "Account Management", "Upgrade", top_k=5))

nodes: 110078
edges: 517739
[('solution:DATA_REPAIR', 0.6649484536082474, 388), ('solution:ENVIRONMENT_ISSUE', 0.6240208877284595, 383), ('solution:DUPLICATE', 0.6222826086956522, 368), ('solution:WORKAROUND', 0.6208955223880597, 335), ('solution:CONFIG_CHANGE', 0.6186666666666667, 375)]


In [5]:
from src.data.graph_rag_query import GraphQuery
##3
query_ticket = Data_tickets[3]

query = GraphQuery(
    text=f"{query_ticket.subject} {query_ticket.description} {query_ticket.error_logs}",
    category=query_ticket.category,
    subcategory=query_ticket.subcategory,
    product=query_ticket.product,
    product_module=query_ticket.product_module,
)
query

GraphQuery(text='License upgrade needed for CloudBackup Enterprise We need to upgrade our license for CloudBackup Enterprise. Our team has grown and we need additional seats. Please provide information on pricing and the upgrade process. 2024-11-27T18:17:26 ERROR ERROR_SERVER_500: Connection timeout after 30s\n2024-11-27T18:17:27 RETRY_FAILED: Max retries exceeded', category='Account Management', subcategory='Upgrade', product='CloudBackup Enterprise', product_module='backup_service')

In [6]:
from src.data.graph_rag_query import graph_rag_candidates

graph_candidates = graph_rag_candidates(
    graph=support_graph,
    query=query,
    top_solutions=5,
    top_error_solutions=5,
    top_tickets=10,
)


In [7]:
graph_candidates

GraphCandidates(solution_nodes=['solution:DUPLICATE', 'solution:DATA_REPAIR', 'solution:ENVIRONMENT_ISSUE', 'solution:CONFIG_CHANGE', 'solution:WORKAROUND', 'solution:PATCH_APPLIED', 'solution:ESCALATED', 'solution:USER_EDUCATION', 'solution:WONT_FIX'], ticket_nodes=['ticket:TK-2024-000002', 'ticket:TK-2024-000004', 'ticket:TK-2024-000031', 'ticket:TK-2024-000092', 'ticket:TK-2024-000172', 'ticket:TK-2024-000186', 'ticket:TK-2024-000205', 'ticket:TK-2024-000271', 'ticket:TK-2024-000304', 'ticket:TK-2024-000334'], solution_priors={'solution:DATA_REPAIR': 0.8232786781428799, 'solution:ENVIRONMENT_ISSUE': 0.7938531745651853, 'solution:DUPLICATE': 1.0, 'solution:WORKAROUND': 0.783653535269434, 'solution:CONFIG_CHANGE': 0.7888420152700604, 'solution:PATCH_APPLIED': 0.4, 'solution:ESCALATED': 0.4, 'solution:USER_EDUCATION': 0.4, 'solution:WONT_FIX': 0.4})

In [8]:
# Issue prior → What usually works for this kind of problem? .
# - comes from top_solutions_for_issue()

# Error boost → What works for THIS specific failure?
# - comes from solutions_from_error_codes()


# solution_priors = {
#   'solution:DATA_REPAIR':        0.8233,
#   'solution:ENVIRONMENT_ISSUE':  0.7939,
#   'solution:DUPLICATE':          1.0,
#   'solution:WORKAROUND':         0.7837,
#   'solution:CONFIG_CHANGE':      0.7888,
#   'solution:PATCH_APPLIED':      0.4,
#   'solution:ESCALATED':          0.4,
#   'solution:USER_EDUCATION':     0.4,
#   'solution:WONT_FIX':           0.4
# }

# Scale meaning

# 0.0 – 0.5 → weak prior (mostly error-based signal)

# 0.6 – 0.85 → strong issue-based prior

# 1.0 → very strong (issue prior + error boost, capped)

In [9]:
from src.rag.milvus_retrieve import connect_milvus,milvus_hybrid_retrieve
client = connect_milvus()
results = milvus_hybrid_retrieve(
    client=client,
    collection_name="support_tickets",
    query_text="App crashes when I open dashboard after login",
    top_k=10,
)
for r in results[:5]:
    print(r['entity']["ticket_id"], r["score"], r['entity'].get("category"), r['entity'].get("resolution_code"))

/home/ramesh/Personal_projects/AI-based-Product-Support-System/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TK-2024-001162 0.016393441706895828 Security CONFIG_CHANGE
TK-2024-006701 0.016393441706895828 Technical Issue WONT_FIX
TK-2024-001101 0.016129031777381897 Security CONFIG_CHANGE
TK-2024-005660 0.016129031777381897 Technical Issue WORKAROUND
TK-2024-000479 0.01587301678955555 Security DATA_REPAIR


In [10]:
results[1]

{'id': 'TK-2024-006701',
 'distance': 0.016393441706895828,
 'entity': {'resolution_helpful': True,
  'ticket_id': 'TK-2024-006701',
  'doc_text': "Analytics Dashboard throwing errors during operation\nWe're experiencing issues with Analytics Dashboard. The system is throwing errors when trying to perform standard operations. This started happening after the recent update. We've tried restarting the service but the issue persists. Our team is blocked and unable to proceed with critical tasks.",
  'category': 'Technical Issue',
  'subcategory': 'Compatibility',
  'product': 'Analytics Dashboard',
  'product_module': 'data_aggregator',
  'resolution_code': 'WONT_FIX'},
 'score': 0.016393441706895828}

In [11]:
query_ticket = Data_tickets[3]


query_text = f"{query_ticket.subject}\n {query_ticket.description} \n {query_ticket.error_logs}"
pred_cat = query_ticket.category
pred_subcat = query_ticket.subcategory
query_text

'License upgrade needed for CloudBackup Enterprise\n We need to upgrade our license for CloudBackup Enterprise. Our team has grown and we need additional seats. Please provide information on pricing and the upgrade process. \n 2024-11-27T18:17:26 ERROR ERROR_SERVER_500: Connection timeout after 30s\n2024-11-27T18:17:27 RETRY_FAILED: Max retries exceeded'

In [12]:
query_ticket

Ticket(ticket_id='TK-2024-000004', created_at=datetime.datetime(2024, 11, 27, 18, 17, 26, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2024, 11, 30, 22, 7, 50, tzinfo=datetime.timezone.utc), customer_id='CUST-04795', customer_tier='starter', organization_id='ORG-231', product='CloudBackup Enterprise', product_version='3.4.15', product_module='backup_service', category='Account Management', subcategory='Upgrade', priority='low', severity='P4', channel='portal', subject='License upgrade needed for CloudBackup Enterprise', description='We need to upgrade our license for CloudBackup Enterprise. Our team has grown and we need additional seats. Please provide information on pricing and the upgrade process.', error_logs='2024-11-27T18:17:26 ERROR ERROR_SERVER_500: Connection timeout after 30s\n2024-11-27T18:17:27 RETRY_FAILED: Max retries exceeded', stack_trace='Stack trace:\n  backup_service::processData() at backup_service.cpp:445\n  Core::runTask() at core.cpp:234\n  main() 

In [13]:
from src.rag.reranker import GraphMilvusReranker

# 1) get milvus results
res = milvus_hybrid_retrieve(client, "support_tickets", query_text, category=pred_cat, subcategory=pred_subcat, top_k=50)

# 2) get graph priors
gc = graph_rag_candidates(support_graph, GraphQuery(text=query_text, category=pred_cat, subcategory=pred_subcat))

# 3) rerank
reranker = GraphMilvusReranker()
final = reranker.rerank(
    milvus_results=res,
    graph_solution_priors=gc.solution_priors,
    pred_category=pred_cat,
    pred_subcategory=pred_subcat,
    top_n=10,
)

print(final[0]["ticket_id"], final[0]["final_score"], final[0]["debug"])


TK-2024-007223 1.7000000000000002 {'milvus_raw': 0.011111111380159855, 'solution_node': 'solution:WONT_FIX', 'helpful': 1.0, 'same_category': 1.0}
